# CSCC11 - Introduction to Machine Learning, Fall 2022, Assignment 1

In [71]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statistics
from scipy import stats

In [72]:
#TO-DO
"""
Read the csv file into a DataFrame - df
"""
df = pd.read_csv('Admission_Predict.csv')

In [73]:
"""
Print the DataFrame
"""
display(df)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [74]:
#TO-DO
"""
Print the length of the DataFrame.
Print the column names of the DataFrame.
"""
print(len(df))
print(df.columns)

400
Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')


In [75]:
#TO-DO
"""
Define an “X” array that would hold our independent features for regression purposes.  
Define a "Y" array that would hold our target variable.

Print the shape of both the arrays.
"""
X = df[['GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research']].to_numpy(copy = True)
print(np.shape(X))

Y = df[['Chance of Admit ']].to_numpy(copy = True)
print(np.shape(Y))

(400, 7)
(400, 1)


## Split the data 

In [76]:
#TO-DO
"""
Split the dataset into train dataset and test dataset.
Set the random state to any number in order to maintain consistency while generating random numbers over several runs.
"""
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=33)

# Linear Regression

In [77]:
#TO-DO
def find_optimal_parameters(x, y):
    """ Compute closed form solution for linear regression!
    Optimal weight w* in linear regression is given by w* = (X^T X)^(-1) X^T Y
    
    Args:
    - x (ndarray (Shape: (N, D))): A NxD matrix corresponding to the inputs.
    - y (ndarray (Shape: (N, 1))): A N-column vector corresponding to the outputs given the inputs.
    
    Output:
    - w (ndarray (Shape: (D, 1))): A D-column vector corresponding to the bias and weights of the linear model.
    """
    # Pad 1's for the bias term, Why?
    # 1 at the front: x = np.hstack((x, np.ones((x.shape[0], 1))))
    x = np.hstack((np.ones((x.shape[0], 1)), x))

    # Note that we could use pseudoinverse here instead: np.linalg.pinv
    # @ is alias for matmul
    p = np.linalg.pinv(x)
    w = np.matmul(p,y)
    return w

### Train linear regression model using training data

In [78]:
#TO-DO
def get_pred_Y(trained_w, X_pred):
    """ Return predicted Y
    Args:
    - trained_w (ndarray (Shape: (D+1, 1))): A (D+1)x1 column vector containing linear regression weights.
    - X_pred (ndarray (Shape: (N, D))): A NxD matrix corresponding to the prediction inputs.
    
    Output:
    - pred_Y (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    """
    pad_Y     = np.hstack((np.ones((X_pred.shape[0],1)), X_pred))
    pred_Y    = pad_Y @ trained_w
    return pred_Y

In [79]:
#TO-DO
def get_mae(Y_truth, Y_pred):
    """ Return Mean absolute error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    if np.shape(Y_truth) == np.shape(Y_pred):
        Y_mean = Y_truth - Y_pred
        mae    = np.mean(np.abs(Y_mean))
        return mae
    return -1

def get_mse(Y_truth, Y_pred):
    """ Return Mean squared error
    Args:
    - Y_truth (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the actual outputs.
    - Y_pred (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the predicted outputs.
    
    Output:
    - MSE (ndarray (Shape: (1,))).
    """
    'check if both inputs are of the same shape'
    if np.shape(Y_truth) == np.shape(Y_pred):
        Y_mean = Y_truth - Y_pred
        mse    = np.mean(np.square(Y_mean))
        return mse
    return -1

### Get predictions on train data

In [80]:
w_optimal = find_optimal_parameters(X_train, Y_train)
print(w_optimal)

[[-1.33382323]
 [ 0.00194302]
 [ 0.00305205]
 [ 0.01160164]
 [-0.01123125]
 [ 0.02232715]
 [ 0.1196734 ]
 [ 0.01890553]]


In [81]:
pred_Y    = get_pred_Y(w_optimal, X_train)
print('train error (MSE): ', get_mse(Y_train, pred_Y))
print('train error (MAE): ', get_mae(Y_train, pred_Y))

train error (MSE):  0.00396475638879563
train error (MAE):  0.04501915140243021


### Get predictions and performance on test data

In [82]:
pred_Y    = get_pred_Y(w_optimal, X_test)
print('test error (MSE):: ', get_mse(Y_test, pred_Y))
print('test error (MAE): ', get_mae(Y_test, pred_Y))

test error (MSE)::  0.004167237997155191
test error (MAE):  0.046087751020785696


# Silouette Coefficient

In [83]:
#TO-DO
n_silhouette = []

kmeans_kwargs= {
    "init":"k-means++",
    "n_init":30,
    "max_iter":250,
    "random_state":2
}

"""
Perform the following steps:

1. Loop over the various possible K values you wish to test
2. Initialize a K means object.
3. Fit the training data on the K means object.
4. Use the silhouette score method available from the sklearn metrics.
5. Append the score to the silhouetter_coefficients list.
6. Display the the silhouette coefficient associated with each value of K.
"""



for n in range(2,11):
    clusters = KMeans(n_clusters = n, init = kmeans_kwargs['init'],
        n_init = kmeans_kwargs['n_init'], max_iter = kmeans_kwargs['max_iter'],
        random_state = kmeans_kwargs["random_state"])
    labels = clusters.fit_predict(X)

    silhouette_avg = silhouette_score(X, labels)
    print("For n_clusters = ", n, ": The average silhouette_score is : ",
        silhouette_avg)
    
    

For n_clusters =  2 : The average silhouette_score is :  0.5233485180906523
For n_clusters =  3 : The average silhouette_score is :  0.45731266107412905
For n_clusters =  4 : The average silhouette_score is :  0.45690921050567296
For n_clusters =  5 : The average silhouette_score is :  0.39334039890325867
For n_clusters =  6 : The average silhouette_score is :  0.3531532855174821
For n_clusters =  7 : The average silhouette_score is :  0.3403140674851437
For n_clusters =  8 : The average silhouette_score is :  0.31526382401888964
For n_clusters =  9 : The average silhouette_score is :  0.3171131560867962
For n_clusters =  10 : The average silhouette_score is :  0.31190098933430455


# K Means

In [84]:
kmeans = KMeans(
    init="k-means++",
    n_clusters = 2, #Input the value you configured using the Silhouette coefficient analysis.
    n_init=30,
    max_iter=250,
    random_state=2
)

#TO-DO
# Fit to the training data
kmeans.fit(X_train)

#TO-DO
# Add the features and the training data you used to the variable below.
training_df_clustered = pd.DataFrame(X_train.copy(), columns = ['GRE Score', 'TOEFL Score',
    'University Rating', 'SOP', 'LOR ', 'CGPA', 'Research'])

#TO-DO
# Predict clusters for the training data
train_cluster = kmeans.predict(X_train)

#TO-DO
# Add the target and predicted clusters to the training DataFrame
training_df_clustered['cluster'] = train_cluster

#TO-DO
# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = 2

X_train_clusters_df = []
for i in range(number_cluster):
    X_train_clusters_df.append(training_df_clustered[training_df_clustered['cluster']==i])

SyntaxError: invalid syntax (4109611177.py, line 16)

# Building Linear Regression for our clusters

In [ ]:
from sklearn.linear_model import LinearRegression

"""
The number of clusters would be defined by the outcome of the silhouetter coefficient 
Set up the model of Linear Regression by exploring the different parameters: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
train_clusters_df is a dataframe that contains both the true cluster values and the predicted cluster values. Feel free to change the variable name to something else if you have been following a different naming convention.
"""

#TO-DO
# Set the number of clusters based on the silhouette coefficient analysis
number_cluster = 2
obj_cluster = []

#TO-DO
# Initialize a Linear Regression object.

for i in range(number_cluster):
    #Get the specific X_train values according to their predicted clusters.
    X_clustered_data = 
    #Get the specific Y_train values according to their predicted clusters.
    Y_clustered_data = 
    obj_cluster.append(__________.fit(X_clustered_data, Y_clustered_data)) #Replace the underlines with the variable name you used to create the Linear Regression object.

In [ ]:
def predict_value(x_test, kmeans, cluster_linear):
  """
  Input: 
  x_test is the test value that you wish to predict on.
  kmeans is the kmeans object that you have finalized to predict on the test dataset.
  cluster_linear is the list of fitted models on different clusters.

  Return:
  linear_pred - linear_pred will be type list with prediction values
  clusters - clusters_pred will be the prediction of clusters using k means.

  Follow these steps:
  1. Predict clusters using K means object on the test data.
  2. Predict regression values using Linear Regression list.
  3. return both the predictions.

  """
  linear_pred = []
  clusters = []
  return linear_pred, clusters

# Final Steps

In [ ]:
#Apply the clustering-based linear regression to the test set.


In [ ]:
print('test error (MSE):: ', get_mse(Y_test, np.array(Y_svr_k_means_pred).reshape(125,1)))
print('test error (MAE): ', get_mae(Y_test, np.array(Y_svr_k_means_pred).reshape(125,1)))